In [1]:
import pandas as pd

ratings_csv = pd.read_csv('../data/raw/RAW_interactions.csv')
recipes_csv = pd.read_csv('../data/raw/RAW_recipes.csv')

ratings_csv.drop_duplicates(inplace=True)
ratings_csv.dropna(inplace=True)


In [2]:
ratings_data = ratings_csv[ratings_csv['date'] >= '2018-01-01']
print(ratings_data.count())

user_id      20585
recipe_id    20585
date         20585
rating       20585
review       20585
dtype: int64


In [3]:
from scipy.sparse import csr_matrix

user_item_matrix = ratings_data.pivot_table(index='user_id', columns='recipe_id', values='rating')

user_item_matrix = user_item_matrix.fillna(0)

sparse_user_item_matrix = csr_matrix(user_item_matrix)




In [4]:
from sklearn.neighbors import NearestNeighbors

recommender = NearestNeighbors(metric='cosine')
recommender.fit(sparse_user_item_matrix)

NearestNeighbors(metric='cosine')

In [5]:
user_id = 2000498330

user_index = user_item_matrix.index.get_loc(user_id)

num_recommendations = 10

_, recommendations = recommender.kneighbors(sparse_user_item_matrix[user_index], n_neighbors=num_recommendations)

recommended_recipes = recipes_csv.iloc[recommendations[0]]['name']

recommended_recipes

2373                             acadia s fruit smoothie 2
13191                      baja black beans  corn and rice
10740    authentic chili verde  pork and green tomatill...
15708                           balsamic and olive chicken
15406          baked tilapia with veggies  herbs  and wine
353                                 company s coming  rice
1575                                 5 minute comfort food
3585                                       almond brownies
195                                     fried  cauliflower
2322      absolutely great  secret  simple mashed potatoes
Name: name, dtype: object